In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
with open("/home/s27mhusa_hpc/Master-Thesis/ner_dataset_sentence.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    print(len(data))

2067


In [3]:
label_list = ["O", "B-soilOrganicCarbon", "I-soilOrganicCarbon", "B-startTime", "I-startTime", "B-endTime", "I-endTime", "B-city", "I-city", "B-duration", "I-duration", "B-cropSpecies", "I-cropSpecies", "B-soilAvailableNitrogen", "I-soilAvailableNitrogen", "B-soilDepth", "I-soilDepth", "B-region", "I-region", "B-country", "I-country", "B-longitude", "I-longitude", "B-latitude", "I-latitude", "B-cropVariety", "I-cropVariety", "B-soilPH", "I-soilPH", "B-soilBulkDensity", "I-soilBulkDensity"]
len(label_list)

31

In [4]:
# Build a label distribution vector for each sentence
def build_label_vector(ner_tags):
    vec = np.zeros(len(label_list), dtype=int)
    for tag in ner_tags:
        if tag != -100:  # Ignore padding
            vec[tag] += 1
    return vec

In [5]:
label_vectors = [build_label_vector(item["ner_tags"]) for item in data]


In [6]:
# Convert label_vectors to a single class by hashing (approx stratification)
label_hashes = [tuple((vec > 0).astype(int)) for vec in label_vectors]
unique_classes = {h: i for i, h in enumerate(set(label_hashes))}
stratify_labels = [unique_classes[h] for h in label_hashes]

In [7]:
from collections import Counter

c = Counter(stratify_labels)
print(len(stratify_labels))
label_grp = 0
for k, v in c.items():
    if v < 2:
        print(f"Label group {k} only occurs {v} time(s)")
        label_grp +=1
print(label_grp)


2067
Label group 13 only occurs 1 time(s)
Label group 17 only occurs 1 time(s)
Label group 49 only occurs 1 time(s)
Label group 54 only occurs 1 time(s)
Label group 31 only occurs 1 time(s)
Label group 81 only occurs 1 time(s)
Label group 52 only occurs 1 time(s)
Label group 25 only occurs 1 time(s)
Label group 9 only occurs 1 time(s)
Label group 8 only occurs 1 time(s)
Label group 19 only occurs 1 time(s)
Label group 38 only occurs 1 time(s)
Label group 74 only occurs 1 time(s)
Label group 76 only occurs 1 time(s)
Label group 72 only occurs 1 time(s)
Label group 77 only occurs 1 time(s)
Label group 3 only occurs 1 time(s)
Label group 34 only occurs 1 time(s)
Label group 45 only occurs 1 time(s)
Label group 5 only occurs 1 time(s)
Label group 36 only occurs 1 time(s)
Label group 32 only occurs 1 time(s)
Label group 20 only occurs 1 time(s)
Label group 1 only occurs 1 time(s)
Label group 58 only occurs 1 time(s)
Label group 33 only occurs 1 time(s)
Label group 28 only occurs 1 time(s)
L

In [8]:
from collections import Counter

# Count how many times each class appears
label_counts = Counter(stratify_labels)

# Keep only samples whose label group appears at least twice
filtered_data = [
    sample for sample, label in zip(data, stratify_labels) if label_counts[label] >= 2
]
filtered_stratify_labels = [
    label for label in stratify_labels if label_counts[label] >= 2
]

# Then split as usual
train_data, temp_data = train_test_split(
    filtered_data,
    stratify=filtered_stratify_labels,
    test_size=0.2,
    random_state=42
)


In [9]:
# Split temp into val and test
val_data, test_data = train_test_split(
    temp_data, test_size=0.5, random_state=42
)

In [10]:
# Save splits
base_path = "/home/s27mhusa_hpc/Master-Thesis"

for split_name, split_data in [("train", train_data), ("val", val_data), ("test", test_data)]:
    out_path = f"{base_path}/ner_dataset_sentence_{split_name}.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(split_data, f, indent=2, ensure_ascii=False)
    print(f"Saved {split_name} split with {len(split_data)} examples to {out_path}")

Saved train split with 1623 examples to /home/s27mhusa_hpc/Master-Thesis/ner_dataset_sentence_train.json
Saved val split with 203 examples to /home/s27mhusa_hpc/Master-Thesis/ner_dataset_sentence_val.json
Saved test split with 203 examples to /home/s27mhusa_hpc/Master-Thesis/ner_dataset_sentence_test.json


/home/s27mhusa_hpc/.conda/envs/Llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'dataset' is not defined